In [1]:
# ////////////////////// 
# 官网教程 输入下面这个网址 能看到超炫的聚类
# tensorboard --logdir=G:\Anaconda3\My_Jupyter\projector\projector
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
# tf.stack的使用 https://www.jianshu.com/p/25706575f8d4
# x = tf.constant([1, 4])
# y = tf.constant([2, 5])
# z = tf.constant([3, 6])
# import tensorflow as tf
# c = tf.stack([x, y, z])  # [[1, 4], [2, 5], [3, 6]] (Pack along first dim.)
# d = tf.stack([x, y, z], axis=1)  # [[1, 2, 3], [4, 5, 6]]
# with tf.Session() as sess:
#     print(sess.run(c))
#     print(sess.run(d))  

In [ ]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)
# 运行次数
max_steps = 1001
# 图片数量
image_num = 3000
# 文件路径
DIR = 'G:/Anaconda3/My_Jupyter'
# 定义会话
sess = tf.Session()
# 载入图片
# 横向打包 mnist数据集的test测试数据集的images图片 [:image_num]从第0张到第image_num = 3000张 打包存放在embedding里
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name = 'embedding')
# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)             # 标准差
        tf.summary.scalar('max',tf.reduce_max(var))    # 最大值
        tf.summary.scalar('min',tf.reduce_min(var))    # 最小值
        tf.summary.histogram('histogram',var)          # 直方图
        
# 命名空间
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 784],name = 'x-input')
    y = tf.placeholder(tf.float32, shape=[None, 10],name = 'y-input')
    
# 显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1,28,28,1])# 把x这个placeholder转化为新形状 -1与None对应 传入几张图不确定 28×28 黑白图 1dim
    tf.summary.image('input',image_shaped_input,10)                                                    # 用这个方法传入10张将新构建的图片
    
with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784, 10]), name = 'W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]), name = 'b')
        variable_summaries(b)
    with tf.name_scope('Wx_plus_b'):
        Wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(Wx_plus_b)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = Wx_plus_b))
    tf.summary.scalar('loss',loss)  # 数据比较多的时候 才需要调用 variable_summaries(var)这个函数 这里 loss只有一个值

with tf.name_scope('train'):
    train = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 
#     train = tf.train.AdamOptimizer(0.001).minimize(loss)

# 初始化变量
sess.run(tf.global_variables_initializer())

# 预测 求准确率
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))# 返回Ture False 求出最大的数在的位置看它俩返回的位置一样不
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))    
        tf.summary.scalar('accuracy',accuracy)      # 算一下准确率的变化
        
        
        
# 产生metadate文件
# 在这个路径下 检查有没有这样的文件 如果有 删掉
# 用写的方式打开这样的文件 其实就是创建文件
# 得到测试集的所有标签 one_hot的最大值所在处就是标签值 保存在labels里
# 循环3000次 写入metadate.tsv
if tf.gfile.Exists(DIR + '/projector/projector/metadate.tsv'):
    tf.gfile.DeleteRecursively(DIR + '/projector/projector/metadate.tsv')
    
with open(DIR + '/projector/projector/metadate.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')


# 合并所有的summary 也就是说 合并所有之前监测的东西
merged = tf.summary.merge_all()                           # 引起血案的地方 jupyter notebook 每次变量不清理 所以每次merge_all前需要restart

# 定义一个writer 把我们要看的graph写入这个路径下
# FileWriter 类提供了一种机制，用于在给定目录中创建事件文件并向其添加摘要和事件
# 该类 异步更新文件内容  这允许训练程序调用方法直接从训练循环向文件添加数据而不会减慢训练速度
projector_writer = tf.summary.FileWriter(DIR + '/projector/projector',sess.graph)

# 保存网络模型
saver = tf.train.Saver()

# 定义配置文件 配置项
config = projector.ProjectorConfig()       

# 把配置项做一些设置
embed = config.embeddings.add()
                
# embedding是打包的载入图片 名字的赋值
embed.tensor_name = embedding.name

# 给他一个 metadate路径的数据 前面生成的metadate
embed.metadata_path = DIR + '/projector/projector/metadate.tsv'

# 给他一个那个大图片的路径
embed.sprite.image_path = DIR + '/projector/data/mnist_10k_sprite.png'

# 把那个大图片按28×28切分     
embed.sprite.single_image_dim.extend([28,28])

# 把这个graph和配置放入 进入可视化       
projector.visualize_embeddings(projector_writer,config)


# 训练
for i in range(max_steps):
    pic_data,pic_tag = mnist.train.next_batch(100)  # 一次训练100张图片 训练1001次
    # 固定的写法 就这样配置
    run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    # 训练一次 统计一次  merged会有一个返回值 存在summary里
    summary,_ =  sess.run([merged,train],feed_dict = {x:pic_data,y:pic_tag},options = run_options,run_metadata = run_metadata)
    # 记录参数变化
    projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
    projector_writer.add_summary(summary,i)
    
#     writer.add_summary(summary,i)
    # 每一次大循环 看一下准确率.
    if i % 100 == 0:
        accuracy_rate = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('第' + str(i) + '次大循环，测试集准确率为' + str(accuracy_rate))
        
# 把训练好的模型保存下来 路径
# 路径的表达 python 默认 /    哈哈哈哈哈哈哈卡了两三天 问题出在这儿
# save_path = saver.save(sess,'.\projector\projector\a_model.ckpt',global_step = max_steps)
save_path = saver.save(sess,'./projector/projector/a_model.ckpt',global_step = max_steps)
print("Model saved in file: %s" % save_path)
projector_writer.close()
sess.close()
tf.logging.set_verbosity(old_v)
                
                
# # 训练完所有图片 即训练完一个大周期 才会打一个点 一共打51个点
# with tf.Session() as sess:
#     sess.run(init)
#     writer = tf.summary.FileWriter('logdir',sess.graph)
#     for epoch in range(51):                                                                                                # 总共训练21次
#         for pic in range(n_batch):                                # 每次训练 是从第n个批次里取1个数据 循环结束 所有批次中的数据被训练一次
#             pic_data,pic_tag = mnist.train.next_batch(batch_size)   # 从train集中选取batch_size个训练数据 next_batch(我觉得这是bagging 错)
#             summary,_ =  sess.run([merged,train],feed_dict={x:pic_data,y:pic_tag}) # 训练一次 统计一次  merged会有一个返回值 存在summary里
#         writer.add_summary(summary,epoch)
#     # 每一次大循环 看一下准确率
#         if epoch % 5 == 0:
#             accuracy_rate = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
#             print('第' + str(epoch) + '次大循环，测试集准确率为' + str(accuracy_rate))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
第0次大循环，测试集准确率为0.1893
第100次大循环，测试集准确率为0.8692
第200次大循环，测试集准确率为0.8844
第300次大循环，测试集准确率为0.8923
第400次大循环，测试集准确率为0.8956
第500次大循环，测试集准确率为0.9006
第600次大循环，测试集准确率为0.9022
